In [ ]:
import numpy as np
from keras.models import Sequential
from keras import applications
from keras import optimizers
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
import csv
import cv2
import scipy
import os

Using TensorFlow backend.


In [ ]:
epochs = 20
# BASE_PATH = '/home/ec2-user/cell_classifier/'
BASE_DIR = '../'
batch_size = 32
num_classes = 4

In [ ]:
def get_data(folder):
    X = []
    y = []

    for wbc_type in os.listdir(folder):
        if not wbc_type.startswith('.'):
#             if wbc_type in ['NEUTROPHIL', 'EOSINOPHIL']:
#                 label = 'MONONUCLEAR'
#             else:
#                 label = 'POLYNUCLEAR'
            for image_filename in os.listdir(folder + wbc_type):
                img_file = cv2.imread(folder + wbc_type + '/' + image_filename)
                img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                if img_file is not None:
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(wbc_type)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [ ]:
X_test, y_test  = get_data(BASE_DIR + '/content/drive/My Drive/Colab Notebooks/Dataset1/TEST/')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  del sys.path[0]


In [ ]:
X_train, y_train = get_data(BASE_DIR + '/content/drive/My Drive/Colab Notebooks/Dataset1/TRAIN/')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  del sys.path[0]


In [ ]:
X_test_simple, y_test_simple = get_data(BASE_DIR + '/content/drive/My Drive/Colab Notebooks/Dataset1/TEST_SIMPLE/')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  del sys.path[0]


In [ ]:
X_train = X_train * 1./255.
X_test = X_test * 1./255.
X_test_simple = X_test_simple * 1./255.

In [ ]:
encoder = LabelEncoder()
encoder.fit(y_test_simple)
y_train = np_utils.to_categorical(encoder.transform(y_train))
y_test = np_utils.to_categorical(encoder.transform(y_test))
y_test_simple = np_utils.to_categorical(encoder.transform(y_test_simple))

In [ ]:

from keras.models import Model
from keras.layers import Input
from keras import optimizers


base_model = applications.VGG16(include_top=False, weights='imagenet')
input = Input(shape=(120, 160, 3),name = 'image_input')
vgg_output = base_model(input)

top_model = Flatten()(vgg_output)
top_model = Dense(64, activation='relu')(top_model)
predictions = Dense(num_classes, activation='softmax', name='prediction_layer')(top_model)

model = Model(input=input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
layers = base_model.layers[:-2]
for layer in layers:
    layer.trainable = False
 
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])



58892288/58889256 [==============================] - 3s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`
  from ipykernel import kernelapp as app


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2, shuffle=True, verbose=1)

Train on 1121 samples, validate on 281 samples
Epoch 1/10
1121/1121 [==============================] - 6s 5ms/step - loss: 0.0822 - accuracy: 0.9911 - val_loss: 2.2702 - val_accuracy: 0.8399
Epoch 2/10
1121/1121 [==============================] - 6s 5ms/step - loss: 0.0094 - accuracy: 0.9991 - val_loss: 3.3194 - val_accuracy: 0.7829
Epoch 3/10
1121/1121 [==============================] - 6s 5ms/step - loss: 0.0034 - accuracy: 0.9991 - val_loss: 3.6012 - val_accuracy: 0.7580
Epoch 4/10
1121/1121 [==============================] - 6s 5ms/step - loss: 0.0095 - accuracy: 0.9991 - val_loss: 2.9398 - val_accuracy: 0.7687
Epoch 5/10
1121/1121 [==============================] - 6s 5ms/step - loss: 0.0056 - accuracy: 0.9991 - val_loss: 5.0479 - val_accuracy: 0.6512
Epoch 6/10
1121/1121 [==============================] - 6s 5ms/step - loss: 0.0137 - accuracy: 0.9991 - val_loss: 4.9050 - val_accuracy: 0.6370
Epoch 7/10
1121/1121 [==============================] - 6s 5ms/step - loss: 0.0046 - accu

In [ ]:
model.save_weights('vgg_top.h5')